In [ ]:
# import general packages, check folders
#%reset
import os
import numpy as np
import h5py
import pandas as pd
#import imagesc as imagesc #pip install imagesc
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import time
from pathlib import Path

# TO DO - REMOVE THE UNNECESSARY LIBRARY IMPORTS

%matplotlib inline

working_dir = '/Users/f0053cz/Dropbox (Dartmouth College)/postdoc_Dartmouth/HCP/fMRIScipts/code'
#working_dir = os.getcwd()
print('current directory:\n',working_dir)
path = Path(working_dir)
parent_folder = path.parent
#print('parent folder:', parent_folder)
data_file_loc = os.path.join(parent_folder,'data') # to store data we extract later in this notebook

In [ ]:
# setting default fontsizes for plots

s=20 # CHANGE FONTSIZE HERE
plt.rc('font', size=s) #controls default text size
plt.rc('axes', titlesize=s) #fontsize of the title
plt.rc('axes', labelsize=s) #fontsize of the x and y labels
plt.rc('xtick', labelsize=s) #fontsize of the x tick labels
plt.rc('ytick', labelsize=s) #fontsize of the y tick labels
plt.rc('legend', fontsize=s) #fontsize of the legend
#import matplotlib as mpl
#mpl.rcParams['font.weight']= 'normal'

In [ ]:
#suff= ' '
suff = '_corrected'

In [ ]:
# load behavioral data - coded 1 for "social", 0 for "nonsocial" and 9 for "unsure", nan for missed response
# even if using the Mental/Random labels, need this to sub-select subs who have responded on all trials
responses = np.load(os.path.join(data_file_loc,f'responses{suff}.npy'))
responses.shape # subs *movies

In [ ]:
vid_joined = ["COAXING-B", "BILLIARD-A", "DRIFTING-A", "Fishing", "Random mechanical","Scaring", "SEDUCING-B", "STAR-A", "SURPRISING-B", "TENNIS-A"]
vid_joined

In [ ]:
print('Nr of complete subs:',len(np.where(np.array([len(np.where(~np.isnan(responses[s,:]))[0]) for s in range(responses.shape[0])])==10)[0]))
print('total subs:', responses.shape[0])

In [ ]:
[timepts_indiv_movie,vid_start_rel_tr] = np.load(os.path.join(data_file_loc,'Video_TRs.npy'),allow_pickle=True)
vid_start_rel_tr

# Run-level regressors vs. traits (option (5) on Miro)

In [ ]:
# import run-level regs
flName = os.path.join(data_file_loc,f'coef_slopereg_all{suff}.npy')
run_level_coefs = np.load(flName)
#run_level_coefs.shape

run_level_coefs_mean = np.nanmean(run_level_coefs, axis=2)
run_level_coefs_mean.shape # USE THIS, BECAUSE THIS WAS USED IN THE MAIN GLM ANALYSIS!

In [ ]:
from pymer4.models import Lm
from multipy.fdr import lsu
from datetime import datetime

print(data_file_loc)
nodes_coaxbill_rand_all = np.load(os.path.join(data_file_loc,'nodes_coaxbill_rand_all.npy')) # sig nodes hihglighted in the first S>NS GLM
# for contours

In [ ]:
data_file_loc = '/Users/f0053cz/Dropbox (Dartmouth College)/postdoc_Dartmouth/HCP/fMRIScipts/data'
sub_id_all = np.load(os.path.join(data_file_loc,f'sub_ID_all{suff}.npy'))
sub_id_all = [str(i) for i in sub_id_all]
sub_id_all = np.array(sub_id_all)
len(sub_id_all)

In [ ]:
vidnames = ["COAXING-B", "BILLIARD-A", "DRIFTING-A", "Fishing", "Random mechanical","Scaring", "SEDUCING-B", "STAR-A", "SURPRISING-B", "TENNIS-A"]

# creating a 3D array of beta coeffs across movies from individual movie files
all_coefs = np.zeros((responses.shape[0],268,10))

for m in range(10):
    fileName =  os.path.join(data_file_loc,'coefs_run_norm','slope_reg',f'coef_slopereg_runnorm_{vidnames[m]}{suff}.npy')
    all_coefs[:,:,m] = np.load(fileName)
    #print(dat.shape)

all_coefs[:10,0,:2]



In [ ]:
# first of all, best to remove subjects with < 10 responses for power in general
count_resp = np.zeros((responses.shape[0],))
for i in range(responses.shape[0]):
    count_resp[i] = len(np.where(~np.isnan(responses[i,:]))[0])
subs_10resp = np.where(count_resp == 10)[0]
len(subs_10resp)

In [ ]:
res_behav_data = pd.read_csv('/Users/f0053cz/Documents/HCP_trait_diffs_DONOTSHARE/trait_analyses/data/RESTRICTED_esfinn_11_21_2021_19_19_35.csv')
res_behav_data.set_index("Subject", inplace=True)
res_behav_data.index = res_behav_data.index.map(str)
print(res_behav_data.shape)
#res_behav_data.head()

# (5) effect of "Social" - "Non-social" beta and trait scores (without considering movies)

In [ ]:
model9_txt = 'coefs_diff ~ ASR_Intn_T'
nsubs_node = []
# LM - run-level regs
model9_res = []
rval_pval_spearman_coef_diff_ASR_Intn_T = []
start_time =  time.time()
for n in range(268):
    if n%10 == 0:
        print(f'node {n+1},time={(time.time()-start_time)/60:.2f} mins')
    
    df = pd.DataFrame({'Subject':sub_id_all[subs_10resp], 'coefs_diff': run_level_coefs_mean[subs_10resp,n], 'subID':subs_10resp})
    df.set_index("Subject", inplace=True)

    df_both = df.join(res_behav_data.loc[:,['ASR_Intn_T']], how='inner') # join betas and trait info
    inds = ~np.isnan(df_both['coefs_diff']) & ~np.isnan(df_both['ASR_Intn_T']) # find rows where neither x or y is NaN
    nsubs_node.append(len(np.where(inds)[0]))
    df_both = df_both.loc[inds,:]
    
    model9 = Lm(model9_txt, data=df_both) # 
    model9.fit(summary = False, verbose = False)
    model9_res.append([model9.coefs['Estimate'],model9.coefs['P-val'],model9.AIC])

    rs,ps = stats.spearmanr(df_both['coefs_diff'],df_both['ASR_Intn_T'])
    rval_pval_spearman_coef_diff_ASR_Intn_T.append([rs,ps])
    if n == 0:
        print('model:',model9.fit(summary = True, verbose = False))

rval_pval_spearman_coef_diff_ASR_Intn_T = np.array(rval_pval_spearman_coef_diff_ASR_Intn_T)
print('Done on/at:',datetime.now()) # cell run at)

In [ ]:
n=31
model9_res[n][0], '1', model9_res[n][1], '2', model9_res[n][2]

In [ ]:
model9_res[n][0][1], '1', model9_res[n][1][1], '2', model9_res[n][2]

In [ ]:
print(model9_txt, model9.formula)
# coef interaction-like term
print(model9_txt)
coef_p_model9 = [[model9_res[n][0][1], model9_res[n][1][1]] for n in range(268)] # ASR reg coefft
coef_p_model9= np.array(coef_p_model9)
print(coef_p_model9[:5,:])

In [ ]:
print("Traits term's dependence on ASR_Intn_T:")
print('nodes with p < .05 :', np.where(coef_p_model9[:,1]<.05)[0])
#coef_p_model9[coef_p_model9[:,1]<.05,1]

coef_p_model9_fdr = lsu(coef_p_model9[:,1],q=.05)#fdr_correction(pval_slope_rand,.05)
print('nodes with q<.05:', np.where(coef_p_model9_fdr)[0])

#glm_nodes_inds = np.where(nodes_coaxbill_rand_all)[0]
#print(f'There are {len(glm_nodes_inds)} GLM nodes.')

#coef_p_model9_fdr_glmnodes = lsu(coef_p_model9[nodes_coaxbill_rand_all,1],q=.05)#fdr_correction(pval_slope_rand,.05)
#print('glm nodes with q<.05 for traits:', glm_nodes_inds[coef_p_model9_fdr_glmnodes])

min(coef_p_model9[:,0]),max(coef_p_model9[:,0])

# Plot

In [ ]:
from nilearn.surface import vol_to_surf
from nilearn.plotting import plot_glass_brain, plot_surf_roi,plot_stat_map,plot_img,plot_surf_contours
import nibabel as nib
from nilearn import datasets
bg_img = datasets.load_mni152_template()
fig_save_loc = os.path.join('/Users/f0053cz/Dropbox (Dartmouth College)/postdoc_Dartmouth/HCP/paper_prep/figures/fig2_2_traits/fmri_results/ALL')

from nilearn.datasets import fetch_surf_fsaverage
fsaverage = fetch_surf_fsaverage()

from nilearn import datasets
bg_img = datasets.load_mni152_template()

from nltools.data import Brain_Data
from nltools.mask import expand_mask, roi_to_brain


mask = Brain_Data('https://neurovault.org/media/images/8423/shen_2mm_268_parcellation.nii.gz')
mask_x = expand_mask(mask)

def color_rois(values):
    """
    This function assumes you are passing a vector "values" with the same length as the number of nodes in the atlas.
    """
    shen268 = nib.load(os.path.join(data_file_loc,"shen_2mm_268_parcellation.nii.gz"))
    shen268_data = shen268.get_fdata()
    img = np.zeros(shen268_data.shape)
    #print(shen268_data.shape)
    for roi in range(len(values)):
        itemindex = np.where(shen268_data==roi+1) # find voxels in this node (add 1 to account for zero-indexing)
        #print(len(itemindex[0]))
        img[itemindex] = values[roi] # color them by the desired value 
    affine = shen268.affine
    img_nii = nib.Nifti1Image(img, affine)
    return img_nii

def surf_plot1(fig,ax,nodes,params, thresh):
    title_txt = params['title']
    txt  = params['txt']
    vmin = params['vmin']
    vmax = params['vmax']

    #LH
    ax_surf = ax[0,0]
    texture = vol_to_surf(color_rois(nodes), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
    surf_plot1=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_left,axes = ax_surf, threshold =  thresh)#)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    texture_contour = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
    plot_surf_contours(fsaverage.infl_left, texture_contour, axes = ax_surf,figure=surf_plot1, legend=True,levels = [1], colors=['k'])

    ax_surf = ax[1,0]
    surf_plot2=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_left, view = 'medial',axes = ax_surf, threshold =  thresh)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    plot_surf_contours(fsaverage.infl_left, texture_contour, axes = ax_surf,figure=surf_plot2, legend=True,levels = [1], colors=['k'])

    #RH
    ax_surf = ax[0,1]
    texture = vol_to_surf(color_rois(nodes), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
    surf_plot3=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=True,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_right,axes = ax_surf, threshold =  thresh)#)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    texture_contour = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
    plot_surf_contours(fsaverage.infl_right, texture_contour, axes = ax_surf,figure=surf_plot3, legend=True,levels = [1], colors=['k'])
    surf_plot3.axes[4].text(10,.5*vmax,s=txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
    box = surf_plot3.axes[4].get_position()
    surf_plot3.axes[4].set_position([box.x0*.93, box.y0-.3, box.width, box.height*2])  # move a bit the bar to the right, need to divide by number of columns (to move relative to last figure only, not to overall row, else will get too far away)
    

    ax_surf = ax[1,1]
    surf_plot4=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=False,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_right, view ='medial',axes = ax_surf, threshold =  thresh)#)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    plot_surf_contours(fsaverage.infl_right, texture_contour, axes = ax_surf,figure=surf_plot4, legend=True, levels = [1], colors=['k'])#, labels=['Sig. (q<.05) across\n(a),(c),(d)'])

    ax[0,0].dist = 7 # change viewing distance to "zoom in" to surface plots
    ax[0,1].dist = 7
    ax[1,0].dist = 7
    ax[1,1].dist = 7

    #fig.colorbar(surf_plot3.axes[2])
    plt.subplots_adjust(left=0,
                        bottom=0, 
                        right=.8, 
                        top=1, 
                        wspace=0.0, 
                        hspace=-.1)    
fig_save_loc

In [ ]:
len(np.where(coef_p_model9[:,1]<.05)[0]), len(np.where((coef_p_model9[:,1]<.05) & (nodes_coaxbill_rand_all))[0])

In [ ]:
min(coef_p_model9[:,0]), max(coef_p_model9[:,0])

In [ ]:
fig_save_loc

In [ ]:
# surface plots
vmin,vmax = -.003,.003
cmap =  'RdBu_r' #'RdBu_r', 'PiYG'
txt = '     'r"$\overline{\beta}(traits)$" # to be corrected - remove overline (NOT "MEAN" BETA)
title_txt = 'traits term beta (p<.05)'
 
thresh = 1e-10
nodes = np.zeros((268,))
nodes[(coef_p_model9[:,1]<.05)] = coef_p_model9[(coef_p_model9[:,1]<.05),0]

fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params,thresh = 1e-10)
plt.savefig(os.path.join(fig_save_loc,f'soc_vs_nonsoc/resp_trait_corr_unc_all_nodes_surf.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img, threshold=thresh,
colorbar= False)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)

plt.savefig(os.path.join(fig_save_loc,f'soc_vs_nonsoc/resp_trait_corr_unc_all_nodes_axial.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

In [ ]:
# surface plots
vmin,vmax = -.003,.003
cmap =  'RdBu_r' #'RdBu_r', 'PiYG'
txt = '     'r"$\overline{\beta}(traits)$" # to be corrected - remove overline (NOT "MEAN" BETA)
title_txt = 'traits term beta (p<.05)'
 
thresh = 1e-10
nodes = np.zeros((268,))
nodes[(coef_p_model9[:,1]<.05) &  (nodes_coaxbill_rand_all)] = coef_p_model9[(coef_p_model9[:,1]<.05) & (nodes_coaxbill_rand_all),0]

fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params,thresh = 1e-10)
plt.savefig(os.path.join(fig_save_loc,f'soc_vs_nonsoc/resp_trait_corr_unc_GLM nodes_surf.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img, threshold=thresh,
colorbar= False)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)

plt.savefig(os.path.join(fig_save_loc,f'soc_vs_nonsoc/resp_trait_corr_unc_GLM nodes_axial.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

### CORRELATION between run-wise diff and traits

In [ ]:
print(np.where(rval_pval_spearman_coef_diff_ASR_Intn_T[:,1]<.05))
print(np.where(lsu(rval_pval_spearman_coef_diff_ASR_Intn_T[:,1])))
print(np.where(lsu(rval_pval_spearman_coef_diff_ASR_Intn_T[nodes_coaxbill_rand_all,1])))


In [ ]:
qcorr = np.zeros((268,))
qcorr[nodes_coaxbill_rand_all] = lsu(rval_pval_spearman_coef_diff_ASR_Intn_T[nodes_coaxbill_rand_all,1])
np.where(qcorr)

min(rval_pval_spearman_coef_diff_ASR_Intn_T[:,0]), max(rval_pval_spearman_coef_diff_ASR_Intn_T[:,0])
rval_pval_spearman_coef_diff_ASR_Intn_T[:,0]


# nodes showing sig (unc p <.05) correlation between "Social"-"Non-social" and trait score
vmin,vmax = -.08,.08
cmap =  'RdBu_r' #'RdBu_r', 'PiYG'
#txt = r"$\overline{\beta}(''S''-''NS'')$"
txt = "rSp_traits" 

title_txt = 'traits term Spearman_r (p<.05)' 
thresh = 1e-10

nodes = np.zeros((268,))
nodes[rval_pval_spearman_coef_diff_ASR_Intn_T[:,1]<.05] = rval_pval_spearman_coef_diff_ASR_Intn_T[rval_pval_spearman_coef_diff_ASR_Intn_T[:,1]<.05,0]

fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params,thresh = 1e-10)
#plt.savefig(os.path.join(fig_save_loc,f'AIC_surf_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img, threshold=thresh,
colorbar= False)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)



print(model9_txt)
coef_p_model9 = [[model9_res[n][0][1], model9_res[n][1][1]] for n in range(268)] # ASR reg coefft
coef_p_model9= np.array(coef_p_model9)
print(coef_p_model9[:5,:])

print('sig terms:', len(np.where(coef_p_model9[:,1]<.05)[0]))
#coef_p_model9[coef_p_model9[:,1]<.05,1]
coef_p_model9_fdr = lsu(coef_p_model9[:,1],q=.05)#fdr_correction(pval_slope_rand,.05)
len(np.where(coef_p_model9_fdr)[0])

min(coef_p_model9[:,0]),max(coef_p_model9[:,0])

# surface plots
vmin,vmax = -.003,.003
cmap =  'RdBu_r' #'RdBu_r', 'PiYG'
txt = r"$\overline{\beta}(traits)$"

title_txt = 'traits term beta (p<.05)'
thresh = 1e-10

nodes = np.zeros((268,))
nodes[coef_p_model9[:,1]<.05] = coef_p_model9[coef_p_model9[:,1]<.05,0]

fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params,thresh = 1e-10)

img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img, threshold=thresh,
colorbar= False)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)

# LME  response label vs. response label + traits

In [ ]:
#install lme functions
from pymer4.models import Lmer
import time


In [ ]:
model1_txt = 'coefs ~  response +(1|subID) + (1|movie)'
model2_txt = 'coefs ~  response*ASR_Intn_T + (1|movie)'
model3_txt = 'coefs ~  response + ASR_Intn_T + (1|movie)'
model4_txt =  'coefs ~ response + (1|movie)'
model5_txt = 'coefs ~  response'
model6_txt = 'coefs ~  response + ASR_Intn_T'
model7_txt = 'coefs ~  response * ASR_Intn_T'
model8_txt = 'coefs ~  ASR_Intn_T'

# Random mech

In [ ]:
# LM - RESPTYPE VS RESPTYPE+TRAIT BASED
load_all = 0 # 1: load saved coefs, 0: run the LME

start_time =  time.time()
model5_res_rand = []
model6_res_rand = []
model7_res_rand= []
model8_res_rand = []

rand_movie_ind = 4

for n in range(268):
    if n%10 == 0:
        print(f'node {n+1},time={(time.time()-start_time)/60:.2f} mins')
    
    df = pd.DataFrame(columns = ['Subject','coefs','subID'])
    

    df_temp = pd.DataFrame({'Subject':sub_id_all, 'coefs': all_coefs[:,n, rand_movie_ind], 'response':responses[:, rand_movie_ind],\
        'subID':np.arange(sub_id_all.shape[0]) })

    df = df.append(df_temp,ignore_index=True)
    #df = df.loc[(df['response'] != 9) & (~np.isnan(df['response'])),:] # yes v no
    df = df.loc[(~np.isnan(df['response'])),:] 
    
    df.set_index("Subject", inplace=True)
    df_both = df.join(res_behav_data.loc[:,['ASR_Intn_T']], how='inner') # join betas and trait info
    inds = ~np.isnan(df_both['coefs']) & ~np.isnan(df_both['ASR_Intn_T']) # find rows where neither x or y is NaN
    df_both = df_both.loc[inds,:]

    nresps_per_sub = df_both.groupby(['subID','response']).count().unstack(level=1).iloc[:,:2]
    nresps_per_sub.columns = nresps_per_sub.columns.droplevel()

    #rows_to_delete = np.zeros((subID.shape[0],))
    #for i in biased_sub_IDs:
    #    rows_to_delete[subID == i] = 1

    #df_both['rows_to_delete'] = rows_to_delete
    df_both_nobiasedsubs = df_both
    #df_both_nobiasedsubs = df_both.loc[df_both['rows_to_delete']==0,:]
    #df_both_nobiasedsubs['response'] = df_both_nobiasedsubs['response'].map({0.0: "Non-social", 1.0:  "Social"}) 
    df_both_nobiasedsubs['response'] = df_both_nobiasedsubs['response'].map({0.0: "Non-social", 1.0:  "XAny_social", 9.0:  "XAny_social"}) # yes or unsure v no
     
    # base LME that depends only on response, taking into account subject and movie variance

    model5 = Lm(model5_txt, data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
    #model1 = Lmer('coefs ~  response +(1|subID) + (1|movie)', data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
    #model5.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)
    #model5.fit(factors={"response":["Non-social","Any_social"]},ordered=True,summary = False, verbose = False)
    model5.fit(summary = False, verbose = False)
    model5_res_rand.append([model5.coefs['Estimate'],model5.coefs['P-val'],model5.AIC])

    # to determine noes with a significant percept-trait interaction
    model6 = Lm(model6_txt, data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
    #model2 = Lmer('coefs ~  response*ASR_Intn_T + (1|movie)', data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
    #model6.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)
    #model6.fit(factors={"response":["Non-social","Any_social"]},ordered=True,summary = False, verbose = False)
    model6.fit(summary = False, verbose = False)
    model6_res_rand.append([model6.coefs['Estimate'],model6.coefs['P-val'],model6.AIC])

    # to determine if the trait term has a sig beta over what response explains
    model7 = Lm(model7_txt, data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
    #model3 = Lmer('coefs ~  response + ASR_Intn_T + (1|movie)', data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
    #model7.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)
    #model7.fit(factors={"response":["Non-social","Any_social"]},ordered=True,summary = False, verbose = False)
    model7.fit(summary = False, verbose = False)
    model7_res_rand.append([model7.coefs['Estimate'],model7.coefs['P-val'],model7.AIC])

    # base LME that depends only on response, taking into account movie variance - for comparison with the above model
    model8 = Lm(model8_txt, data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
    #model4 = Lmer('coefs ~  response + (1|movie)', data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
    #model8.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)
    #model8.fit(factors={"response":["Non-social","Any_social"]},ordered=True,summary = False, verbose = False)
    model8.fit(summary = False, verbose = False)
    model8_res_rand.append([model8.coefs['Estimate'],model8.coefs['P-val'],model8.AIC])

    if n == 0:
        #print('model:',model5.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False))
        #print('model:',model5.fit(factors={"response":["Non-social","Social"]}))
        print('model:',model5.fit(factors={"response":["Non-social","Any_social"]},summary = False, verbose = False))
        print('model:',model5.fit(factors={"response":["Non-social","Any_social"]}))

print('Done on/at:',datetime.now()) # cell run at)

# All movies

In [ ]:
# LMER - RESPTYPE VS RESPTYPE+TRAIT BASED
load_all = 0 # 1: load saved coefs, 0: run the LME


#if load_all == 1:
#     df_all_coefs = pd.read_csv(os.path.join(data_file_loc,'coefs_run_norm','slope_reg','LME_S_NS_revision1','lmecoefs_all_S_NS_S_NSwithtraits.csv'))
#[coef_ALL_stim,pval_ALL_stim] = np.load(os.path.join(data_file_loc,'coefs_run_norm','slope_reg','LME_S_NS_revision1','lmecoefs_all_S_NS_stimType.npy'), allow_pickle=True)
#else:
start_time =  time.time()
model1_res = []
model2_res = []
model3_res = []
model4_res = []
for n in range(268):
    if n%10 == 0:
        print(f'node {n+1},time={(time.time()-start_time)/60:.2f} mins')
    
    df = pd.DataFrame(columns = ['Subject','coefs','subID','movie'])
    for m in range(10):
        df_temp = pd.DataFrame({'Subject':sub_id_all[subs_10resp], 'coefs': all_coefs[subs_10resp,n,m], 'response':responses[subs_10resp,m],\
            'subID':subs_10resp,\
                'movie':np.repeat(m,len(subs_10resp))})
        df = df.append(df_temp,ignore_index=True)
        df = df.loc[df['response'] != 9] # yes v no
    
    df.set_index("Subject", inplace=True)
    df_both = df.join(res_behav_data.loc[:,['ASR_Intn_T']], how='inner') # join betas and trait info
    inds = ~np.isnan(df_both['coefs']) & ~np.isnan(df_both['ASR_Intn_T']) # find rows where neither x or y is NaN
    df_both = df_both.loc[inds,:]

    nresps_per_sub = df_both.groupby(['subID','response']).count().unstack(level=1).iloc[:,:2]
    nresps_per_sub.columns = nresps_per_sub.columns.droplevel()

    biased_sub_IDs1 = list(nresps_per_sub[nresps_per_sub[0].isna()].index)
    biased_sub_IDs1 # No "Non-social response"

    biased_sub_IDs2 = list(nresps_per_sub[nresps_per_sub[1].isna()].index)
    biased_sub_IDs2 #  No "Social" response

    biased_sub_IDs = biased_sub_IDs1 + biased_sub_IDs2

    subID = df_both['subID'].values
    rows_to_delete = np.zeros((subID.shape[0],))
    for i in biased_sub_IDs:
        rows_to_delete[subID == i] = 1

    df_both['rows_to_delete'] = rows_to_delete
    df_both_nobiasedsubs = df_both.loc[df_both['rows_to_delete']==0,:]
    df_both_nobiasedsubs['response'] = df_both_nobiasedsubs['response'].map({0.0: "Non-social", 1.0:  "Social"})
    
    # base LME that depends only on response, taking into account subject and movie variance
    model1 = Lmer(model1_txt, data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
    #model1 = Lmer('coefs ~  response +(1|subID) + (1|movie)', data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
    model1.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)
    model1_res.append([model1.coefs['Estimate'],model1.coefs['P-val'],model1.AIC])

    # to determine noes with a significant percept-trait interaction
    model2 = Lmer(model2_txt, data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
    #model2 = Lmer('coefs ~  response*ASR_Intn_T + (1|movie)', data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
    model2.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)
    model2_res.append([model2.coefs['Estimate'],model2.coefs['P-val'],model2.AIC])

    # to determine if the trait term has a sig beta over what response explains
    model3 = Lmer(model3_txt, data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
    #model3 = Lmer('coefs ~  response + ASR_Intn_T + (1|movie)', data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
    model3.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)
    model3_res.append([model3.coefs['Estimate'],model3.coefs['P-val'],model3.AIC])

    # base LME that depends only on response, taking into account movie variance - for comparison with the above model
    model4 = Lmer(model4_txt, data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
    #model4 = Lmer('coefs ~  response + (1|movie)', data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
    model4.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)
    model4_res.append([model4.coefs['Estimate'],model4.coefs['P-val'],model4.AIC])

    #if n == 0:
    #    print('model:',model1.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False))
    #    print('model:',model2.fit(factors={"response":["Non-social","Social"]}))


print('Done on/at:',datetime.now()) # cell run at)

In [ ]:
# LM - RESP VS RESPTYPE+TRAIT BASED

'''load_all = 0 # 1: load saved coefs, 0: run the LME
if load_all == 1:
    df_all_coefs = pd.read_csv(os.path.join(data_file_loc,'coefs_run_norm','slope_reg','LME_S_NS_revision1','lmecoefs_all_S_NS_S_NSwithtraits.csv'))
    #[coef_ALL_stim,pval_ALL_stim] = np.load(os.path.join(data_file_loc,'coefs_run_norm','slope_reg','LME_S_NS_revision1','lmecoefs_all_S_NS_stimType.npy'), allow_pickle=True)
else:'''

model5_res = []
model6_res = []
model7_res = []
model8_res = []
start_time =  time.time()
for n in range(268):
    if n%10 == 0:
        print(f'node {n+1},time={(time.time()-start_time)/60:.2f} mins')
    
    df = pd.DataFrame(columns = ['Subject','coefs','subID','movie'])
    for m in range(10):
        df_temp = pd.DataFrame({'Subject':sub_id_all[subs_10resp], 'coefs': all_coefs[subs_10resp,n,m], 'response':responses[subs_10resp,m],\
            'subID':subs_10resp,\
                'movie':np.repeat(m,len(subs_10resp))})
        df = df.append(df_temp,ignore_index=True)
        df = df.loc[df['response'] != 9] # yes v no
    
    df.set_index("Subject", inplace=True)
    df_both = df.join(res_behav_data.loc[:,['ASR_Intn_T']], how='inner') # join betas and trait info
    inds = ~np.isnan(df_both['coefs']) & ~np.isnan(df_both['ASR_Intn_T']) # find rows where neither x or y is NaN
    df_both = df_both.loc[inds,:]

    nresps_per_sub = df_both.groupby(['subID','response']).count().unstack(level=1).iloc[:,:2]
    nresps_per_sub.columns = nresps_per_sub.columns.droplevel()

    biased_sub_IDs1 = list(nresps_per_sub[nresps_per_sub[0].isna()].index)
    biased_sub_IDs1 # No "Non-social response"

    biased_sub_IDs2 = list(nresps_per_sub[nresps_per_sub[1].isna()].index)
    biased_sub_IDs2 #  No "Social" response

    biased_sub_IDs = biased_sub_IDs1 + biased_sub_IDs2

    subID = df_both['subID'].values
    rows_to_delete = np.zeros((subID.shape[0],))
    for i in biased_sub_IDs:
        rows_to_delete[subID == i] = 1

    df_both['rows_to_delete'] = rows_to_delete
    df_both_nobiasedsubs = df_both.loc[df_both['rows_to_delete']==0,:]
    df_both_nobiasedsubs['response'] = df_both_nobiasedsubs['response'].map({0.0: "Non-social", 1.0:  "Social"})

    model5 = Lm(model5_txt, data=df_both_nobiasedsubs) # 
    model5.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)
    model5_res.append([model5.coefs['Estimate'],model5.coefs['P-val'],model5.AIC])
    
    model6 = Lm(model6_txt, data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
    model6.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)
    model6_res.append([model6.coefs['Estimate'],model6.coefs['P-val'],model6.AIC])

    model7 = Lm(model7_txt, data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
    model7.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)
    model7_res.append([model7.coefs['Estimate'],model7.coefs['P-val'],model7.AIC])

    model8 = Lm(model8_txt, data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
    model8.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)
    model8_res.append([model8.coefs['Estimate'],model8.coefs['P-val'],model8.AIC])

    #if n == 0:
    #    print('model:',model1.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False))
    #    print('model:',model2.fit(factors={"response":["Non-social","Social"]}))

print('Done on/at:',datetime.now()) # cell run at)

## Unsure response trials

In [ ]:
# LM - Unsure trials

model8_res_unsure = []
start_time =  time.time()
for n in range(268):
    if n%10 == 0:
        print(f'node {n+1},time={(time.time()-start_time)/60:.2f} mins')
    
    df = pd.DataFrame(columns = ['Subject','coefs','subID','movie'])
    for m in range(10):
        df_temp = pd.DataFrame({'Subject':sub_id_all[subs_10resp], 'coefs': all_coefs[subs_10resp,n,m], 'response':responses[subs_10resp,m],\
            'subID':subs_10resp,\
                'movie':np.repeat(m,len(subs_10resp))})
        df = df.append(df_temp,ignore_index=True)
        df = df.loc[df['response'] == 9] # only "Unsure"
    
    df.set_index("Subject", inplace=True)
    df_both = df.join(res_behav_data.loc[:,['ASR_Intn_T']], how='inner') # join betas and trait info
    inds = ~np.isnan(df_both['coefs']) & ~np.isnan(df_both['ASR_Intn_T']) # find rows where neither x or y is NaN
    df_both = df_both.loc[inds,:]


    subID = df_both['subID'].values
    rows_to_delete = np.zeros((subID.shape[0],))
    for i in biased_sub_IDs:
        rows_to_delete[subID == i] = 1

    df_both_nobiasedsubs = df_both
    #df_both_nobiasedsubs['response'] = df_both_nobiasedsubs['response'].map({0.0: "Non-social", 1.0:  "Social"})

    model8 = Lm(model8_txt, data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
    model8.fit(summary = False, verbose = False)
    model8_res_unsure.append([model8.coefs['Estimate'],model8.coefs['P-val'],model8.AIC])

    #if n == 0:
    #    print('model:',model1.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False))
    #    print('model:',model2.fit(factors={"response":["Non-social","Social"]}))

print('Done on/at:',datetime.now()) # cell run at)

In [ ]:
model8_txt

In [ ]:
model8_txt

In [ ]:
model10_txt =  'coefs ~ stimType * ASR_Intn_T'

In [ ]:
# LM - Unsure trials

model10_res_unsure = []
start_time =  time.time()
for n in range(268):
    if n%10 == 0:
        print(f'node {n+1},time={(time.time()-start_time)/60:.2f} mins')
    
    df = pd.DataFrame(columns = ['Subject','coefs','subID','movie'])
    for m in range(10):
        df_temp = pd.DataFrame({'Subject':sub_id_all[subs_10resp], 'coefs': all_coefs[subs_10resp,n,m], 'response':responses[subs_10resp,m],\
            'subID':subs_10resp,\
                'movie':np.repeat(m,len(subs_10resp))})
        df = df.append(df_temp,ignore_index=True)
        df = df.loc[df['response'] == 9] # only "Unsure"
    
    df.set_index("Subject", inplace=True)
    df_both = df.join(res_behav_data.loc[:,['ASR_Intn_T']], how='inner') # join betas and trait info
    inds = ~np.isnan(df_both['coefs']) & ~np.isnan(df_both['ASR_Intn_T']) # find rows where neither x or y is NaN
    df_both = df_both.loc[inds,:]


    subID = df_both['subID'].values
    rows_to_delete = np.zeros((subID.shape[0],))
    for i in biased_sub_IDs:
        rows_to_delete[subID == i] = 1

    df_both_nobiasedsubs = df_both
    df_both_nobiasedsubs['stimType'] = ['Mental' if i < 5 else "ARandom" for i in df_both_nobiasedsubs['movie'] ] 
    #df_both_nobiasedsubs['response'] = df_both_nobiasedsubs['response'].map({0.0: "Non-social", 1.0:  "Social"})

    model10 = Lm(model10_txt, data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
    model10.fit(summary = False, verbose = False)
    model10_res_unsure.append([model10.coefs['Estimate'],model10.coefs['P-val'],model10.AIC])

    #if n == 0:
    #    print('model:',model1.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False))
    #    print('model:',model2.fit(factors={"response":["Non-social","Social"]}))

print('Done on/at:',datetime.now()) # cell run at)

# Plot results on brain

In [ ]:
print('Model 1:', model1.formula, '\nModel 2:', model2.formula, '\nModel 3:', model3.formula, '\nModel 4:', model4.formula, 
'\nModel 5:', model5.formula, '\nModel 6:', model6.formula, '\nModel 7:', model7.formula, '\nModel 8:', model8.formula)

# (1) AIC model 3 v. model 4 (LME models with and without the traits term)

In [ ]:
print('model3:',model3_txt, '\nmodel4:', model4_txt)

In [ ]:
AIC_diff =  np.array([model3_res[n][2] -  model4_res[n][2] for n in range(268)])
plt.hist(AIC_diff)
np.where(AIC_diff < -2)[0]

In [ ]:
shen268_lbl = pd.read_csv(os.path.join(data_file_loc,"shen_dictionary.csv"))
pd.set_option('display.max_colwidth', None)
shen268_lbl.head()

In [ ]:
n=49
eval(shen268_lbl[str(n+1)][0])['name'], eval(shen268_lbl[str(n+1)][0])['coords']

In [ ]:
min(AIC_diff),max(AIC_diff)

In [ ]:
# surface plots
vmin,vmax = -23,23
cmap =  'PiYG' #'RdBu_r', 'PiYG'
txt = ' AIC Traitsmodel-\nNoTraitsModel'
#txt = r' $\beta_{TraitScore}FDR.$'
#txt = r' $\overline{\beta}(\""")-$' + '\n' + r'$\overline{\beta}_{Non-social}$'+ '\n'
#txt = '' #'AIC response-\nbased LME - \nstim-based LME' #'AIC S/NS LME - M/R LME'#r"$\overline{\beta}{(diff)}$"

title_txt = 'Internalizing score reg coefft (p<.05)'

nodes = np.zeros((268,))
nodes = AIC_diff
#nodes[coef_p_model3_intn_fdr] = coef_p_model3_intn[coef_p_model3_intn_fdr,0]

thresh = 2 #1e-10
fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params,thresh = thresh)
#plt.savefig(os.path.join(fig_save_loc,f'AIC_surf_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img, threshold= thresh,
colorbar= False)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)

#plt.savefig(os.path.join(fig_save_loc,f'AIC_axial_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

## (1a) trait term coefft - model 3

In [ ]:
model3_txt

In [ ]:
# coef trait term
coef_p_model3_traits = [[model3_res[n][0][2], model3_res[n][1][2]] for n in range(268)]
coef_p_model3_traits = np.array(coef_p_model3_traits)
print(coef_p_model3_traits[:5,:])

Traits_sig_ROIs = np.where(coef_p_model3_traits[:,1]<.05)[0]
Traits_sig_ROIs


In [ ]:
coef_p_model3_traits_fdr = lsu(coef_p_model3_traits[:,1],q=.05)#fdr_correction(pval_slope_rand,.05)
np.where(coef_p_model3_traits_fdr)[0], len(np.where(coef_p_model3_traits_fdr)[0])

In [ ]:
vmin,vmax = min(coef_p_model3_traits[:,0]),max(coef_p_model3_traits[:,0])
vmin,vmax

In [ ]:
# surface plots
vmin,vmax = -.005,.005
cmap =  'RdBu_r' #'RdBu_r', 'PiYG'
txt = r' $\beta_{TraitScore}Unc.$'
#txt = r' $\beta_{TraitScore}FDR.$'
#txt = r' $\overline{\beta}(\""")-$' + '\n' + r'$\overline{\beta}_{Non-social}$'+ '\n'
#txt = '' #'AIC response-\nbased LME - \nstim-based LME' #'AIC S/NS LME - M/R LME'#r"$\overline{\beta}{(diff)}$"

title_txt = 'Internalizing score reg coefft (p<.05)'
 
thresh = 1e-10

nodes = np.zeros((268,))
nodes[coef_p_model3_traits[:,1]<.05] = coef_p_model3_traits[coef_p_model3_traits[:,1]<.05,0]

thresh = 1e-10
fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params,thresh = 1e-10)
#plt.savefig(os.path.join(fig_save_loc,f'AIC_surf_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img, threshold= thresh,
colorbar= False)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)

#plt.savefig(os.path.join(fig_save_loc,f'AIC_axial_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

# model 3 vs. 4 - comparing response beta terms between a model with and without trait terms - does adding a trait term for nodes showing trait sensitivity increase beta?

In [ ]:
model3_txt, model4_txt

In [ ]:
model3_res[49][0], model4_res[49][0]

In [ ]:
beta_resp_model3 = np.array([model3_res[n][0][1] for n in range(268)])
beta_resp_model4 = np.array([model4_res[n][0][1] for n in range(268)])

plt.hist(beta_resp_model3, color='blue', alpha = .3)
plt.hist(beta_resp_model4, color='green', alpha = .3)

In [ ]:
plt.hist(beta_resp_model3[beta_resp_model3 >beta_resp_model4]-beta_resp_model4[beta_resp_model3 >beta_resp_model4], color='blue')
plt.hist(beta_resp_model3[beta_resp_model3 < beta_resp_model4] - beta_resp_model4[beta_resp_model3 < beta_resp_model4], color='green')


In [ ]:
beta_diff34 = beta_resp_model3-beta_resp_model4
plt.hist(beta_diff34, color='grey')
plt.hist(beta_diff34[coef_p_model3_traits_fdr], color='red')
stats.ttest_1samp(beta_diff34,0), np.nanmean(beta_diff34)

In [ ]:
beta_diff34_abs =  np.abs(beta_resp_model3) - np.abs(beta_resp_model4)
plt.hist(beta_diff34_abs, color='grey')
plt.hist(beta_diff34_abs[coef_p_model3_traits_fdr], color='red')
stats.ttest_1samp(beta_diff34_abs,0), np.nanmean(beta_diff34_abs)

In [ ]:
print('No traits model. min and max model 4:', min(beta_resp_model4[coef_p_model3_traits_fdr]),max(beta_resp_model4[coef_p_model3_traits_fdr]))

In [ ]:
# surface plots
vmin,vmax = -.4,.4
cmap =  'RdBu_r' #'RdBu_r', 'PiYG'
txt = 'responseBeta\n(NoTraitsModel)'
#txt = r' $\overline{\beta}(\""")-$' + '\n' + r'$\overline{\beta}_{Non-social}$'+ '\n'
#txt = '' #'AIC response-\nbased LME - \nstim-based LME' #'AIC S/NS LME - M/R LME'#r"$\overline{\beta}{(diff)}$"
 
thresh = 1e-10

nodes = np.zeros((268,))
nodes[coef_p_model3_traits_fdr] = beta_resp_model4[coef_p_model3_traits_fdr]

fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params,thresh = 1e-10)
#plt.savefig(os.path.join(fig_save_loc,f'AIC_surf_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img, threshold=thresh,
colorbar= False)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)

#plt.savefig(os.path.join(fig_save_loc,f'AIC_axial_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

In [ ]:
print('with traits model. min and max model 3:', min(beta_resp_model3[coef_p_model3_traits_fdr]), max(beta_resp_model3[coef_p_model3_traits_fdr]))

In [ ]:
# surface plots
vmin,vmax = -.4,.4
cmap =  'RdBu_r' #'RdBu_r', 'PiYG'
txt = 'responseBeta\n(Traitsmodel)'
#txt = r' $\overline{\beta}(\""")-$' + '\n' + r'$\overline{\beta}_{Non-social}$'+ '\n'
#txt = '' #'AIC response-\nbased LME - \nstim-based LME' #'AIC S/NS LME - M/R LME'#r"$\overline{\beta}{(diff)}$"
 
thresh = 1e-10

nodes = np.zeros((268,))
nodes[coef_p_model3_traits_fdr] = beta_resp_model3[coef_p_model3_traits_fdr]

fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params,thresh = 1e-10)
#plt.savefig(os.path.join(fig_save_loc,f'AIC_surf_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img, threshold=thresh,
colorbar= False)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)

#plt.savefig(os.path.join(fig_save_loc,f'AIC_axial_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

# (6) "Unsure" response analyses

In [ ]:
model8_txt

In [ ]:
# coef trait term
coef_p_model8_traits_unsure = [[model8_res_unsure[n][0][1], model8_res_unsure[n][1][1]] for n in range(268)]
coef_p_model8_traits_unsure = np.array(coef_p_model8_traits_unsure)
print(coef_p_model8_traits_unsure[:5,:])

Traits_sig_ROIs = np.where(coef_p_model8_traits_unsure[:,1]<.05)[0]
Traits_sig_ROIs

coef_p_model8_traits_unsure_fdr = lsu(coef_p_model8_traits_unsure[:,1],q=.05)#fdr_correction(pval_slope_rand,.05)
np.where(coef_p_model8_traits_unsure_fdr)[0], len(np.where(coef_p_model8_traits_unsure[:,1]<.05)[0])

In [ ]:
min(coef_p_model8_traits_unsure[:,0]), max(coef_p_model8_traits_unsure[:,0])

In [ ]:
np.where(coef_p_model8_traits_unsure_fdr[nodes_coaxbill_rand_all])

In [ ]:
# surface plots
vmin,vmax = -.01,.01
cmap =  'RdBu_r' #'RdBu_r', 'PiYG'
txt = 'Est_traits_Unsure'
#txt = r' $\overline{\beta}(\""")-$' + '\n' + r'$\overline{\beta}_{Non-social}$'+ '\n'
#txt = '' #'AIC response-\nbased LME - \nstim-based LME' #'AIC S/NS LME - M/R LME'#r"$\overline{\beta}{(diff)}$"
 
thresh = 1e-10

nodes = np.zeros((268,))
nodes[coef_p_model8_traits_unsure[:,1]<.05] = coef_p_model8_traits_unsure[coef_p_model8_traits_unsure[:,1]<.05,0]

fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params,thresh = 1e-10)

img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img, threshold=thresh,
colorbar= False)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)


# (b) stimType*traits

In [ ]:
# coef trait term
coef_p_model10_traits_unsure = [[model10_res_unsure[n][0][3], model10_res_unsure[n][1][3]] for n in range(268)]
coef_p_model10_traits_unsure = np.array(coef_p_model10_traits_unsure)
print(coef_p_model10_traits_unsure[:5,:])

Traits_sig_ROIs = np.where(coef_p_model10_traits_unsure[:,1]<.05)[0]
Traits_sig_ROIs

coef_p_model10_traits_unsure_fdr = lsu(coef_p_model10_traits_unsure[:,1],q=.05)#fdr_correction(pval_slope_rand,.05)
np.where(coef_p_model10_traits_unsure_fdr)[0], len(np.where(coef_p_model10_traits_unsure[:,1]<.05)[0])

In [ ]:
model10.design_matrix

In [ ]:
np.where(lsu(coef_p_model10_traits_unsure[nodes_coaxbill_rand_all,1],q=.05))

In [ ]:
min(coef_p_model10_traits_unsure[:,0]), max(coef_p_model10_traits_unsure[:,0])

In [ ]:
np.where(coef_p_model10_traits_unsure_fdr)

In [ ]:
# surface plots
vmin,vmax = -.02,.02
cmap =  'RdBu_r' #'RdBu_r', 'PiYG'
txt = 'Est_Interaction term_Unsure'
#txt = r' $\overline{\beta}(\""")-$' + '\n' + r'$\overline{\beta}_{Non-social}$'+ '\n'
#txt = '' #'AIC response-\nbased LME - \nstim-based LME' #'AIC S/NS LME - M/R LME'#r"$\overline{\beta}{(diff)}$"
 
thresh = 1e-10

nodes = np.zeros((268,))
nodes[coef_p_model10_traits_unsure[:,1]<.05] = coef_p_model10_traits_unsure[coef_p_model10_traits_unsure[:,1]<.05,0]

fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params,thresh = 1e-10)

img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img, threshold=thresh,
colorbar= False)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)


In [ ]:
np.where(coef_p_model10_traits_unsure[:,1]<.05)[0]

In [ ]:
n = 0

df = pd.DataFrame(columns = ['Subject','coefs','subID','movie'])
for m in range(10):
    df_temp = pd.DataFrame({'Subject':sub_id_all[subs_10resp], 'coefs': all_coefs[subs_10resp,n,m], 'response':responses[subs_10resp,m],\
        'subID':subs_10resp, 'movie':np.repeat(m,len(subs_10resp))})
    df = df.append(df_temp,ignore_index=True)
    df = df.loc[df['response'] == 9] # only "Unsure"

df.set_index("Subject", inplace=True)
df_both = df.join(res_behav_data.loc[:,['ASR_Intn_T']], how='inner') # join betas and trait info
inds = ~np.isnan(df_both['coefs']) & ~np.isnan(df_both['ASR_Intn_T']) # find rows where neither x or y is NaN
df_both = df_both.loc[inds,:]


subID = df_both['subID'].values
rows_to_delete = np.zeros((subID.shape[0],))
for i in biased_sub_IDs:
    rows_to_delete[subID == i] = 1

df_both_nobiasedsubs = df_both
df_both_nobiasedsubs['stimType'] = ['Mental' if i < 5 else "Random" for i in df_both_nobiasedsubs['movie'] ] 

In [ ]:
mean_per_sub = df_both_nobiasedsubs.groupby(['subID','stimType']).mean().unstack(level=1)
mean_per_sub.columns =  ['_'.join(col) for col in mean_per_sub.columns.values] #mean_per_sub.columns.droplevel()
mean_per_sub

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
sns.regplot(x = 'ASR_Intn_T_Mental',     y = 'coefs_Mental', data = mean_per_sub, color = 'k', label = '"Mental"', ax=ax, marker = '.')
sns.regplot(x = 'ASR_Intn_T_Random',     y = 'coefs_Random', data = mean_per_sub, color = [.6,.6,.6], label = '"Random"', ax=ax, marker = '.')

rs1, ps1 = stats.spearmanr(mean_per_sub['ASR_Intn_T_Mental'], mean_per_sub['coefs_Mental'], nan_policy='omit')
rs2, ps2 = stats.spearmanr(mean_per_sub['ASR_Intn_T_Random'], mean_per_sub['coefs_Random'], nan_policy='omit')
print(f'"Mental":{rs1:.4f},(p={ps1:.4f})')
print(f'"Random":{rs2:.4f},(p={ps2:.4f})')
#plt.annotate(r"$ r_{S}= -.03$"f' (p='r"$.63)$", xy=(.3,.1), xycoords='axes fraction',fontsize=20, color = 'k') # 0.1, .003
#plt.annotate(r"$ r_{S}=  .19$"f' (p='r"$.017)$", xy=(.3,.01), xycoords='axes fraction',fontsize=20, color = [.7,.7,.7]) # 0.1, .003
plt.xlabel('Internalizing Score (T)')
plt.ylabel('Mean beta Estimate')
plt.legend(bbox_to_anchor = [1.01,1])
plt.ylim(-3,3)
plt.xlim(29,100)

In [ ]:
# EDIT FOR UNSURE RESPONSE TRIALS
# surface plots
vmin,vmax = -.4,.4
cmap =  'RdBu_r' #'RdBu_r', 'PiYG'
txt = 'responseBeta\n(Traitsmodel)'
#txt = r' $\overline{\beta}(\""")-$' + '\n' + r'$\overline{\beta}_{Non-social}$'+ '\n'
#txt = '' #'AIC response-\nbased LME - \nstim-based LME' #'AIC S/NS LME - M/R LME'#r"$\overline{\beta}{(diff)}$"
 
thresh = 1e-10

nodes = np.zeros((268,))
nodes[coef_p_model3_traits_fdr] = beta_resp_model3[coef_p_model3_traits_fdr]

fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params,thresh = 1e-10)
#plt.savefig(os.path.join(fig_save_loc,f'AIC_surf_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img, threshold=thresh,
colorbar= False)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)

#plt.savefig(os.path.join(fig_save_loc,f'AIC_axial_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

In [ ]:
print('min mag. diff:', min( np.abs(beta_resp_model3[coef_p_model3_traits_fdr] - beta_resp_model4[coef_p_model3_traits_fdr])))
print('max mag. diff:', max( np.abs(beta_resp_model3[coef_p_model3_traits_fdr] - beta_resp_model4[coef_p_model3_traits_fdr])))

In [ ]:
100*(beta_resp_model3[coef_p_model3_traits_fdr] - beta_resp_model4[coef_p_model3_traits_fdr])/(beta_resp_model4[coef_p_model3_traits_fdr])

In [ ]:
coef_pval_traitsig_nodes = np.array([[model4_res[n][0][1],model4_res[n][1][1]] for n in np.where(coef_p_model3_traits_fdr)[0]])
coef_pval_traitsig_nodes, coef_pval_traitsig_nodes.shape

In [ ]:
len(np.where(coef_pval_traitsig_nodes[:,1]<.05)[0])

In [ ]:
plt.scatter(coef_pval_traitsig_nodes[:,0],coef_pval_traitsig_nodes[:,1])

In [ ]:
# surface plots
vmin,vmax = -.01,.01
cmap =  'RdBu_r' #'RdBu_r', 'PiYG'
txt = 'responseBeta\n(Traits-NoTraits)'
#txt = r' $\overline{\beta}(\""")-$' + '\n' + r'$\overline{\beta}_{Non-social}$'+ '\n'
#txt = '' #'AIC response-\nbased LME - \nstim-based LME' #'AIC S/NS LME - M/R LME'#r"$\overline{\beta}{(diff)}$"

title_txt = 'beta diff (with traits vs. without in the sig nodes)'
 
thresh = 1e-10

nodes = np.zeros((268,))
nodes[coef_p_model3_traits_fdr] = beta_resp_model3[coef_p_model3_traits_fdr] - beta_resp_model4[coef_p_model3_traits_fdr]

fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params,thresh = 1e-10)
#plt.savefig(os.path.join(fig_save_loc,f'AIC_surf_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img, threshold=thresh,
colorbar= False)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)

#plt.savefig(os.path.join(fig_save_loc,f'AIC_axial_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

- blue: magnitude of response increases in model 3 !
e.g.

In [ ]:
beta_resp_model3[coef_p_model3_intn_fdr][0], beta_resp_model4[coef_p_model3_intn_fdr][0]

In [ ]:
min(beta_resp_model3 - beta_resp_model4),max(beta_resp_model3 - beta_resp_model4)

In [ ]:
# surface plots
vmin,vmax = -.01,.01
cmap =  'RdBu_r' #'RdBu_r', 'PiYG'
txt = 'responseBeta\n(Traits-NoTraits)'
#txt = r' $\overline{\beta}(\""")-$' + '\n' + r'$\overline{\beta}_{Non-social}$'+ '\n'
#txt = '' #'AIC response-\nbased LME - \nstim-based LME' #'AIC S/NS LME - M/R LME'#r"$\overline{\beta}{(diff)}$"

title_txt = 'beta diff (with traits vs. without in the sig nodes)'
 
thresh = 1e-10

nodes = np.zeros((268,))
nodes = beta_resp_model3 - beta_resp_model4

fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params,thresh = 1e-10)
#plt.savefig(os.path.join(fig_save_loc,f'AIC_surf_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img, threshold=thresh,
colorbar= False)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)

#plt.savefig(os.path.join(fig_save_loc,f'AIC_axial_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

# (2) response * trait interaction - model 2

In [ ]:
# coef interaction term
print(model2_txt)
coef_p_model2 = [[model2_res[n][0][3], model2_res[n][1][3]] for n in range(268)] # ASR_response_interaction_term
coef_p_model2= np.array(coef_p_model2)
print(coef_p_model2[:5,:])
print('sig terms:', )

coef_p_model2_fdr = lsu(coef_p_model2[:,1],q=.05)#fdr_correction(pval_slope_rand,.05)
np.where(coef_p_model2_fdr)[0]
# min(coef_p_model2[coef_p_model2[:,1]<.05,0]), max(coef_p_model2[coef_p_model2[:,1]<.05,0])

# surface plots
vmin,vmax = -.0055,.0055
cmap =  'RdBu_r' #'RdBu_r', 'PiYG'
txt = r' $\beta_{InteractionTerm}$'
#txt = r' $\overline{\beta}(\""")-$' + '\n' + r'$\overline{\beta}_{Non-social}$'+ '\n'
#txt = '' #'AIC response-\nbased LME - \nstim-based LME' #'AIC S/NS LME - M/R LME'#r"$\overline{\beta}{(diff)}$"

title_txt = 'Interaction term beta (p<.05)'
 
thresh = 1e-10

nodes = np.zeros((268,))
nodes[coef_p_model2[:,1]<.05] = coef_p_model2[coef_p_model2[:,1]<.05,0]

fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params,thresh = 1e-10)
#plt.savefig(os.path.join(fig_save_loc,f'AIC_surf_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img, 
colorbar= False)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)

#plt.savefig(os.path.join(fig_save_loc,f'AIC_axial_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

# (3) Linear reg - effect of adding traits term

In [ ]:
print('model5_txt:',model5_txt, '\nmodel6_txt:',model6_txt)

In [ ]:
AIC_LM_diff_addingAIC = np.array([model6_res[n][2] - model5_res[n][2] for n in range(268)])
plt.hist(AIC_LM_diff_addingAIC)
AIC_LM_diff_addingAIC.mean()

In [ ]:
print('model5_txt:',model5_txt, '\nmodel6_txt:',model6_txt)
AIC_LM_diff_addingAIC_RAND = np.array([model6_res_rand[n][2] - model5_res_rand[n][2] for n in range(268)])
plt.hist(AIC_LM_diff_addingAIC_RAND)
AIC_LM_diff_addingAIC_RAND.mean()


In [ ]:
nodes_that_improve =  np.where((AIC_LM_diff_addingAIC<-2))[0]
len(nodes_that_improve)


In [ ]:
nodes_that_improve_rand =  np.where((AIC_LM_diff_addingAIC_RAND<-2))[0]
len(nodes_that_improve_rand)

In [ ]:
min(AIC_LM_diff_addingAIC), max(AIC_LM_diff_addingAIC)

In [ ]:
model5_txt,model6_txt

In [ ]:
# all movies

n = 49 # the MT-like node
df = pd.DataFrame(columns = ['Subject','coefs','subID','movie'])
for m in range(10):
    df_temp = pd.DataFrame({'Subject':sub_id_all[subs_10resp], 'coefs': all_coefs[subs_10resp,n,m], 'response':responses[subs_10resp,m],\
        'subID':subs_10resp,\
            'movie':np.repeat(m,len(subs_10resp))})
    df = df.append(df_temp,ignore_index=True)
    df = df.loc[df['response'] != 9] # yes v no

df.set_index("Subject", inplace=True)
df_both = df.join(res_behav_data.loc[:,['ASR_Intn_T']], how='inner') # join betas and trait info
inds = ~np.isnan(df_both['coefs']) & ~np.isnan(df_both['ASR_Intn_T']) # find rows where neither x or y is NaN
df_both = df_both.loc[inds,:]

nresps_per_sub = df_both.groupby(['subID','response']).count().unstack(level=1).iloc[:,:2]
nresps_per_sub.columns = nresps_per_sub.columns.droplevel()

biased_sub_IDs1 = list(nresps_per_sub[nresps_per_sub[0].isna()].index)
biased_sub_IDs1 # No "Non-social response"

biased_sub_IDs2 = list(nresps_per_sub[nresps_per_sub[1].isna()].index)
biased_sub_IDs2 #  No "Social" response

biased_sub_IDs = biased_sub_IDs1 + biased_sub_IDs2

subID = df_both['subID'].values
rows_to_delete = np.zeros((subID.shape[0],))
for i in biased_sub_IDs:
    rows_to_delete[subID == i] = 1

df_both['rows_to_delete'] = rows_to_delete
df_both_nobiasedsubs = df_both.loc[df_both['rows_to_delete']==0,:]
df_both_nobiasedsubs['response'] = df_both_nobiasedsubs['response'].map({0.0: "Non-social", 1.0:  "Social"})

In [ ]:
rand_movie_ind

In [ ]:
mean_per_sub = df_both_nobiasedsubs.groupby(['subID','response']).mean().unstack(level=1)
mean_per_sub

In [ ]:
mean_per_sub.columns =  ['_'.join(col) for col in mean_per_sub.columns.values] #mean_per_sub.columns.droplevel()
mean_per_sub

In [ ]:
import seaborn as sns

#red_rgb = [103,0,31] # edges of RdBu
#blue_rgb = [5,48,97] # edges of RdBu
red_rgb =[188,61,62] # from Emily
blue_rgb = [54,122,177] # from Emily
red_rgb = np.array(red_rgb)/255
blue_rgb = np.array(blue_rgb)/255
alpha = .2 # transparency inside boxplots, for datapts etc.

In [ ]:
def explore_correlation(x, y, data):
    """
    Calculates and plots correlation between x and y variables in dataframe `data`, plus distribution of x and y 
    """
    sns.set_style("white")
    
    inds = ~np.isnan(data[x]) & ~np.isnan(data[y]) # find rows where neither x or y is NaN

    g = sns.jointplot(x=x, y=y, data=data, kind='reg', color='gray')

    # Calculate and print correlations
    rp, pp = stats.pearsonr(data[x][inds], data[y][inds])
    rs, ps = stats.spearmanr(data[x][inds], data[y][inds])
    #g.ax_joint.annotate(f'r_s = {rs:.2f}\n(p={ps:.1e})', xy=(.05,.8), xycoords='axes fraction')
    print(f'Spearman r={rs}, p = {ps}')
    
    return g

In [ ]:
mean_per_sub['abs_coefs_Non-social'] = np.abs(mean_per_sub['coefs_Non-social'])
mean_per_sub['abs_coefs_Social'] = np.abs(mean_per_sub['coefs_Social'])
mean_per_sub

In [ ]:
explore_correlation(x = 'ASR_Intn_T_Non-social', y = 'abs_coefs_Non-social', data = mean_per_sub)
explore_correlation(x = 'ASR_Intn_T_Social', y = 'abs_coefs_Social', data = mean_per_sub)

#explore_correlation(x = 'ASR_Intn_T_Non-social', y = 'coefs_Non-social', data = mean_per_sub)
#explore_correlation(x = 'ASR_Intn_T_Social', y = 'coefs_Social', data = mean_per_sub)


In [ ]:
def independent_corr(xy, ab, n, n2 = None, twotailed=True, conf_level=0.95, method='fisher'):
    """
    Calculates the statistic significance between two independent correlation coefficients
    @param xy: correlation coefficient between x and y
    @param xz: correlation coefficient between a and b
    @param n: number of elements in xy
    @param n2: number of elements in ab (if distinct from n)
    @param twotailed: whether to calculate a one or two tailed test, only works for 'fisher' method
    @param conf_level: confidence level, only works for 'zou' method
    @param method: defines the method uses, 'fisher' or 'zou'
    @return: z and p-val
    """

    if method == 'fisher':
        xy_z = 0.5 * np.log((1 + xy)/(1 - xy))
        ab_z = 0.5 * np.log((1 + ab)/(1 - ab))
        if n2 is None:
            n2 = n

        se_diff_r = np.sqrt(1/(n - 3) + 1/(n2 - 3))
        diff = xy_z - ab_z
        z = abs(diff / se_diff_r)
        p = (1 - norm.cdf(z))
        if twotailed:
            p *= 2

        return z, p
    elif method == 'zou':
        L1 = rz_ci(xy, n, conf_level=conf_level)[0]
        U1 = rz_ci(xy, n, conf_level=conf_level)[1]
        L2 = rz_ci(ab, n2, conf_level=conf_level)[0]
        U2 = rz_ci(ab, n2, conf_level=conf_level)[1]
        lower = xy - ab - pow((pow((xy - L1), 2) + pow((U2 - ab), 2)), 0.5)
        upper = xy - ab + pow((pow((U1 - xy), 2) + pow((ab - L2), 2)), 0.5)
        return lower, upper
    else:
        raise Exception('Wrong method!')

#print(dependent_corr(.40, .50, .10, 103, method='steiger'))
#print(independent_corr(0.5 , 0.6, 103, 103, method='fisher'))


In [ ]:
# The following function was taken from here: https://github.com/psinger/CorrelationStats/blob/master/corrstats.py

from scipy.stats import t, norm
from math import atanh, pow
from numpy import tanh

def rz_ci(r, n, conf_level = 0.95):
    zr_se = pow(1/(n - 3), .5)
    moe = norm.ppf(1 - (1 - conf_level)/float(2)) * zr_se
    zu = atanh(r) + moe
    zl = atanh(r) - moe
    return tanh((zl, zu))

def rho_rxy_rxz(rxy, rxz, ryz):
    num = (ryz-1/2.*rxy*rxz)*(1-pow(rxy,2)-pow(rxz,2)-pow(ryz,2))+pow(ryz,3)
    den = (1 - pow(rxy,2)) * (1 - pow(rxz,2))
    return num/float(den)

def dependent_corr(xy, xz, yz, n, twotailed=True, conf_level=0.95, method='steiger'):
    """
    Calculates the statistic significance between two dependent correlation coefficients
    @param xy: correlation coefficient between x and y
    @param xz: correlation coefficient between x and z
    @param yz: correlation coefficient between y and z
    @param n: number of elements in x, y and z
    @param twotailed: whether to calculate a one or two tailed test, only works for 'steiger' method
    @param conf_level: confidence level, only works for 'zou' method
    @param method: defines the method uses, 'steiger' or 'zou'
    @return: t and p-val
    """
    if method == 'steiger':
        d = xy - xz
        determin = 1 - xy * xy - xz * xz - yz * yz + 2 * xy * xz * yz
        av = (xy + xz)/2
        cube = (1 - yz) * (1 - yz) * (1 - yz)

        t2 = d * np.sqrt((n - 1) * (1 + yz)/(((2 * (n - 1)/(n - 3)) * determin + av * av * cube)))
        p = 1 - t.cdf(abs(t2), n - 3)

        if twotailed:
            p *= 2

        return t2, p
    elif method == 'zou':
        L1 = rz_ci(xy, n, conf_level=conf_level)[0]
        U1 = rz_ci(xy, n, conf_level=conf_level)[1]
        L2 = rz_ci(xz, n, conf_level=conf_level)[0]
        U2 = rz_ci(xz, n, conf_level=conf_level)[1]
        rho_r12_r13 = rho_rxy_rxz(xy, xz, yz)
        lower = xy - xz - pow((pow((xy - L1), 2) + pow((U2 - xz), 2) - 2 * rho_r12_r13 * (xy - L1) * (U2 - xz)), 0.5)
        upper = xy - xz + pow((pow((U1 - xy), 2) + pow((xz - L2), 2) - 2 * rho_r12_r13 * (U1 - xy) * (xz - L2)), 0.5)
        return lower, upper
    else:
        raise Exception('Wrong method!')

In [ ]:
r,p = stats.spearmanr(mean_per_sub['ASR_Intn_T_Social'], mean_per_sub['coefs_Social'])
print('Social:',r,p)
r,p = stats.spearmanr(mean_per_sub['ASR_Intn_T_Non-social'], mean_per_sub['coefs_Non-social'])
print('Non-social:',r,p)


In [ ]:
xy = stats.spearmanr(mean_per_sub['ASR_Intn_T_Social'], mean_per_sub['coefs_Social'])[0]
xz = stats.spearmanr(mean_per_sub['ASR_Intn_T_Social'], mean_per_sub['coefs_Non-social'])[0]
yz = stats.spearmanr(mean_per_sub['coefs_Social'], mean_per_sub['coefs_Non-social'])[0]

inds = ~np.isnan(mean_per_sub['ASR_Intn_T_Social']) & ~np.isnan(mean_per_sub['coefs_Social']) & ~np.isnan(mean_per_sub['coefs_Non-social'])
print(inds.sum())

dependent_corr(xy, xz, yz, n, twotailed=False)

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
sns.regplot(x = 'ASR_Intn_T_Social',     y = 'coefs_Social', data = mean_per_sub, color = red_rgb, label = '"Social"', ax=ax, marker = '.')
sns.regplot(x = 'ASR_Intn_T_Non-social', y = 'coefs_Non-social', data = mean_per_sub, color = blue_rgb, label = '"Non-social"', ax=ax, marker = '.')
plt.annotate(r"$ r_{S}=-.07$"f' (p='r"$.051)$", xy=(.3,.1), xycoords='axes fraction',fontsize=20, color = red_rgb) # 0.1, .003
plt.annotate(r"$ r_{S}=-.09$"f' (p='r"$.009)$", xy=(.3,.01), xycoords='axes fraction',fontsize=20, color = blue_rgb) # 0.1, .003
plt.xlabel('Internalizing Score (T)')
plt.ylabel('Mean LME Estimate')
plt.legend(bbox_to_anchor = [1.01,1])
plt.ylim(-3,3)
plt.xlim(29,100)

### ONLY RAND

In [ ]:
min(AIC_LM_diff_addingAIC_RAND),max(AIC_LM_diff_addingAIC_RAND)

In [ ]:
# surface plots
vmin,vmax = -9,9
cmap = 'PiYG' #'RdBu_r'
txt = 'AIC diff on\nadding traits term- RAND'
#txt = r' $\overline{\beta}(\""")-$' + '\n' + r'$\overline{\beta}_{Non-social}$'+ '\n'
title_txt = ''
 
nodes = np.zeros((268,))
nodes = AIC_LM_diff_addingAIC_RAND
thresh = 2

fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params, thresh)
#plt.savefig(os.path.join(fig_save_loc,f'AIC_surf_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
    
img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img,colorbar= False,threshold = thresh)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)

#plt.savefig(os.path.join(fig_save_loc,f'AIC_axial_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

In [ ]:
model5_txt, model6_txt

In [ ]:
coefs_rand_resp = np.array([model5_res_rand[n][0][1] for n in range(268)])
pval_rand_resp = np.array([model5_res_rand[n][1][1] for n in range(268)])

min(coefs_rand_resp[pval_rand_resp<.05]), max(coefs_rand_resp[pval_rand_resp<.05])

In [ ]:
# surface plots
vmin,vmax = -.3, .3
cmap = 'RdBu_r' # 'PiYG'
txt = 'estimate_response in the LM:\ncoefs ~ response (RAND)'
#txt = r' $\overline{\beta}(\""")-$' + '\n' + r'$\overline{\beta}_{Non-social}$'+ '\n'
title_txt = ''
 
nodes = np.zeros((268,))
nodes[pval_rand_resp<.05] = coefs_rand_resp[pval_rand_resp<.05]
thresh = 1e-6

fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params, thresh)
#plt.savefig(os.path.join(fig_save_loc,f'AIC_surf_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
    
img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img,colorbar= False,threshold = thresh)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)

#plt.savefig(os.path.join(fig_save_loc,f'AIC_axial_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

In [ ]:
coefs_rand_resp = np.array([model6_res_rand[n][0][1] for n in range(268)])
pval_rand_resp = np.array([model6_res_rand[n][1][1] for n in range(268)])
pval_rand_resp_fdr = lsu(pval_rand_resp,q=.05)
print(len(np.where(pval_rand_resp<.05)[0]), len(np.where(pval_rand_resp_fdr)[0]))

min(coefs_rand_resp[pval_rand_resp<.05]), max(coefs_rand_resp[pval_rand_resp<.05])

In [ ]:
# surface plots
vmin,vmax = -.3, .3
cmap = 'RdBu_r' # 'PiYG'
txt = 'estimate_response in the LM:\ncoefs ~ response + ASR (RAND)'
#txt = r' $\overline{\beta}(\""")-$' + '\n' + r'$\overline{\beta}_{Non-social}$'+ '\n'
title_txt = ''
 
nodes = np.zeros((268,))
nodes[pval_rand_resp<.05] = coefs_rand_resp[pval_rand_resp<.05]
thresh = 1e-6

fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params, thresh)
#plt.savefig(os.path.join(fig_save_loc,f'AIC_surf_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
    
img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img,colorbar= False,threshold = thresh)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)

#plt.savefig(os.path.join(fig_save_loc,f'AIC_axial_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

In [ ]:
model7_res_rand[0][0]

In [ ]:
coefs_rand_interaction = np.array([model7_res_rand[n][0][3] for n in range(268)])
pval_rand_interaction = np.array([model7_res_rand[n][1][3] for n in range(268)])
pval_rand_interaction_fdr = lsu(pval_rand_interaction,q=.05)

print(len(np.where(pval_rand_interaction<.05)[0]), len(np.where(pval_rand_interaction_fdr)[0]))

min(coefs_rand_interaction[pval_rand_interaction<.05]), max(coefs_rand_interaction[pval_rand_interaction<.05])

In [ ]:
np.where(lsu(pval_rand_interaction[nodes_coaxbill_rand_all],q=.05))[0]

In [ ]:
# surface plots
vmin,vmax = -.03, .03
cmap = 'RdBu_r' # 'PiYG'
txt = 'estimate_response in the LM:\ncoefs ~ response * ASR (RAND)'
#txt = r' $\overline{\beta}(\""")-$' + '\n' + r'$\overline{\beta}_{Non-social}$'+ '\n'
title_txt = ''
 
nodes = np.zeros((268,))
nodes[pval_rand_interaction<.05] = coefs_rand_interaction[pval_rand_interaction<.05]
thresh = 1e-6

fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params, thresh)
#plt.savefig(os.path.join(fig_save_loc,f'AIC_surf_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
    
img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img,colorbar= False,threshold = thresh)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)

#plt.savefig(os.path.join(fig_save_loc,f'AIC_axial_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

In [ ]:
# surface plots
'''vmin,vmax = -.3, .3
cmap = 'RdBu_r' # 'PiYG'
txt = 'estimate_response in the LM:\ncoefs ~ response + ASR (RAND)'
#txt = r' $\overline{\beta}(\""")-$' + '\n' + r'$\overline{\beta}_{Non-social}$'+ '\n'
title_txt = ''
 
nodes = np.zeros((268,))
nodes[pval_rand_resp<.05] = coefs_rand_resp[pval_rand_resp<.05]
thresh = 1e-6

fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params, thresh)
#plt.savefig(os.path.join(fig_save_loc,f'AIC_surf_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
    
img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img,colorbar= False,threshold = thresh)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)

#plt.savefig(os.path.join(fig_save_loc,f'AIC_axial_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()'''

In [ ]:
'''coefs_rand_resp5 = np.array([model5_res_rand[n][0][1] for n in range(268)])
pval_rand_resp5 = np.array([model5_res_rand[n][1][1] for n in range(268)])
pval_rand_resp5_fdr = lsu(pval_rand_resp5,q=.05)
pval_rand_resp5_fdr_glm = lsu(pval_rand_resp5[nodes_coaxbill_rand_all],q=.05)

coefs_rand_resp6 = np.array([model6_res_rand[n][0][1] for n in range(268)])
pval_rand_resp6 = np.array([model6_res_rand[n][1][1] for n in range(268)])
pval_rand_resp6_fdr = lsu(pval_rand_resp6,q=.05)
pval_rand_resp6_fdr_glm = lsu(pval_rand_resp6[nodes_coaxbill_rand_all],q=.05)

plt.hist(coefs_rand_resp5[pval_rand_resp6<.05] - coefs_rand_resp6[pval_rand_resp6<.05]) # beta with ASR - without ASR'''


In [ ]:
max(coefs_rand_interaction[pval_rand_interaction<.05])
np.where(coefs_rand_interaction==max(coefs_rand_interaction))

In [ ]:
np.where(pval_rand_interaction == min(pval_rand_interaction))

In [ ]:
lsu(pval_rand_interaction,.05)[89]

In [ ]:
np.where(np.where(nodes_coaxbill_rand_all)[0]==89)

In [ ]:
# random mech
n = 89 # 49 is the MT-like node
df = pd.DataFrame(columns = ['Subject','coefs','subID'])

df_temp = pd.DataFrame({'Subject':sub_id_all, 'coefs': all_coefs[:,n,rand_movie_ind], 'response':responses[:,rand_movie_ind],\
    'subID':np.arange(sub_id_all.shape[0]) })
df = df.append(df_temp,ignore_index=True)
#df = df.loc[(df['response'] != 9) & ~np.isnan(df['response']),:] # yes v no
df = df.loc[~np.isnan(df['response']),:] # yes v no


df.set_index("Subject", inplace=True)
df_both_RAND = df.join(res_behav_data.loc[:,['ASR_Intn_T']], how='inner') # join betas and trait info
inds = ~np.isnan(df_both_RAND['coefs']) & ~np.isnan(df_both_RAND['ASR_Intn_T']) # find rows where neither x or y is NaN
df_both_RAND = df_both_RAND.loc[inds,:]
df_both_RAND['response'] = df_both_RAND['response'].map({0.0: "Non-social", 1.0:  "Any_social", 9.0:  "Any_social"})
df_both_RAND.head(),df_both_RAND.shape

df_both_RAND_soc = df_both_RAND.loc[df_both_RAND['response']=="Any_social",:]
df_both_RAND_nonsoc = df_both_RAND.loc[df_both_RAND['response']=="Non-social",:]

print('"Any_social"')
explore_correlation(x = 'ASR_Intn_T', y = 'coefs', data = df_both_RAND_soc)
print('"Non-social"')
explore_correlation(x = 'ASR_Intn_T', y = 'coefs', data = df_both_RAND_nonsoc)

x = 'ASR_Intn_T'
y= 'coefs'
#rp, pp = stats.pearsonr(df_both_RAND_soc[x],    df_both_RAND_soc[y])
#rp, pp = stats.pearsonr(df_both_RAND_nonsoc[x], df_both_RAND_nonsoc[y])
rs1, ps1 = stats.spearmanr(df_both_RAND_soc[x],    df_both_RAND_soc[y])
rs2, ps2 = stats.spearmanr(df_both_RAND_nonsoc[x], df_both_RAND_nonsoc[y])

print(rs1, rs2)

z, p = independent_corr(rs1 , rs2, df_both_RAND_soc.shape[0], df_both_RAND_nonsoc.shape[0], method='fisher')
print('z=',z, ',p=', p ) # not sig different

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
sns.regplot(x = 'ASR_Intn_T',     y = 'coefs', data = df_both_RAND_soc, color = red_rgb, label = '"Social" or "Unsure"', ax=ax, marker = '.')
sns.regplot(x = 'ASR_Intn_T', y = 'coefs', data = df_both_RAND_nonsoc, color = blue_rgb, label = '"Non-social"', ax=ax, marker = '.')
rs1, ps1 = stats.spearmanr(df_both_RAND_soc[x],    df_both_RAND_soc[y])
rs2, ps2 = stats.spearmanr(df_both_RAND_nonsoc[x], df_both_RAND_nonsoc[y])
print(f'"Social":{rs1:.4f},(p={ps1:.4f})')
print(f'"Non-social":{rs2:.4f},(p={ps2:.4f})')
plt.annotate(r"$ r_{S}=  .12$"f' (p='r"$.03)$", xy=(.3,.1), xycoords='axes fraction',fontsize=20, color = red_rgb) # 0.1, .003
plt.annotate(r"$ r_{S}= -.06$"f' (p='r"$.13)$", xy=(.3,.01), xycoords='axes fraction',fontsize=20, color = blue_rgb) # 0.1, .003
plt.xlabel('Internalizing Score (T)')
plt.ylabel('LME Estimate')
plt.legend(bbox_to_anchor = [1.01,1])
plt.ylim(-3,3)
plt.xlim(29,100)

### ALL MOVIES

In [ ]:
# surface plots
vmin,vmax = -33,33
cmap = 'PiYG' #'RdBu_r'
txt = 'AIC diff on\nadding traits term'
#txt = r' $\overline{\beta}(\""")-$' + '\n' + r'$\overline{\beta}_{Non-social}$'+ '\n'
title_txt = ''
 
nodes = np.zeros((268,))
nodes = AIC_LM_diff_addingAIC
thresh = 2

fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params, thresh)
#plt.savefig(os.path.join(fig_save_loc,f'AIC_surf_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
    
img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img,colorbar= False,threshold = thresh)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)

#plt.savefig(os.path.join(fig_save_loc,f'AIC_axial_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

# Looking at nodes which show trait dependence

In [ ]:
model8_txt, model8.formula

In [ ]:
coefs_traits =  np.array([[model8_res[n][0][1],model8_res[n][1][1]] for n in range(268)])
coefs_traits[:5,:]

In [ ]:
coefs_traits_fdr = lsu(coefs_traits[:,1],q=.05)#fdr_correction(pval_slope_rand,.05)
np.where(coefs_traits_fdr)[0]

In [ ]:
len(np.where(coefs_traits[:,1] < .05)[0])

In [ ]:
min(coefs_traits[:,0]), max(coefs_traits[:,0])

In [ ]:
# surface plots
vmin,vmax = -.005,.005
cmap = 'RdBu_r' #'RdBu_r', 'PiYG'
#txt = r' $\overline{\beta}_{traits}Unc$'
txt = r' $\overline{\beta}_{traits}FDR$'
title_txt = ''
 
nodes = np.zeros((268,))
#nodes[coefs_traits[:,1]<.05] = coefs_traits[coefs_traits[:,1]<.05,0]
nodes[coefs_traits_fdr] = coefs_traits[coefs_traits_fdr,0]
thresh = 1e-10

fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params, thresh)
#plt.savefig(os.path.join(fig_save_loc,f'AIC_surf_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
    
img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img,colorbar= False,threshold = thresh)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)

#plt.savefig(os.path.join(fig_save_loc,f'AIC_axial_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

In [ ]:
model8.formula

# extra code

In [ ]:
'''nresps_per_sub = df_both.groupby(['subID','response']).count().unstack(level=1).iloc[:,:2]
nresps_per_sub.columns = nresps_per_sub.columns.droplevel()

biased_sub_IDs1 = list(nresps_per_sub[nresps_per_sub["Non-social"].isna()].index)
biased_sub_IDs1 # No "Non-social response"

biased_sub_IDs2 = list(nresps_per_sub[nresps_per_sub["Social"].isna()].index)
biased_sub_IDs2 #  No "Social" response

biased_sub_IDs = biased_sub_IDs1 + biased_sub_IDs2

subID = df_both['subID'].values
rows_to_delete = np.zeros((subID.shape[0]))
for i in biased_sub_IDs:
    rows_to_delete[subID == i] = 1

df_both['rows_to_delete'] = rows_to_delete
df_both_nobiasedsubs = df_both.loc[df_both['rows_to_delete']==0,:]
print(df_both_nobiasedsubs.shape, df_both.shape)

model1 = Lmer('coefs ~  response +(1|subID) + (1|movie)', data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
model1.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)

model2 = Lmer('coefs ~  response + ASR_Intn_T + (1|subID) + (1|movie)', data=df_both) # ff: mean response, rf:subjID
model2.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)

'''

'''data = pd.DataFrame({})#columns = ['Subject','coef','response','movie'])
n=0
nresp = [len(np.where(~np.isnan(responses[i,:]))[0]) for i in range(responses.shape[0])]
for m in range(10):
    #fMRI_data = pd.DataFrame({'coefs': all_coefs[subs_10resp,n,m],'response':responses[subs_10resp,m], 'subID':sub_id_all[subs_10resp], 'movie':np.repeat(m,len(subs_10resp))})
    fMRI_data = pd.DataFrame({'Subject':sub_id_all,'slopeReg_node': all_coefs_run[:,n,m], 'response': responses[:,m],
    'nresp':nresp, 'movie': np.repeat(m,1049)})
    #fMRI_data = fMRI_data.loc[fMRI_data['nresp']==10,:]
    fMRI_data.set_index("Subject", inplace=True)
    data_temp = fMRI_data.join(behav_data.loc[:,['ASR_Intn_T','Age_in_Yrs','Gender']], how='inner') # join betas and trait info
    data = data.append(data_temp)#,ignore_index=True)
data.reset_index(inplace=True)

inds = ~np.isnan(data['slopeReg_node']) # find rows with missing fMRI data
data = data.loc[inds,:]
data.shape'''

In [ ]:
'''coef_ALL_resp, coef_ALL_resp_traits, coef_ALL_resp_traits_nosubID , coef_ALL_resp_traits_nosubID_notraits= [np.empty((268,)) for _ in range(4)]
coef_ALL_resp[:], coef_ALL_resp_traits[:],coef_ALL_resp_traits_nosubID[:],coef_ALL_resp_traits_nosubID_notraits[:] = [np.nan]*4

pval_ALL_resp, pval_ALL_resp_traits,pval_ALL_resp_traits_nosubID,pval_ALL_resp_traits_nosubID_notraits = [np.empty((268,)) for _ in range(4)]
pval_ALL_resp[:], pval_ALL_resp_traits[:], pval_ALL_resp_traits_nosubID[:],pval_ALL_resp_traits_nosubID_notraits[:]  = [np.nan]*4

AIC_ALL_resp, AIC_ALL_resp_traits, AIC_ALL_resp_traits_nosubID, AIC_ALL_resp_traits_nosubID_notraits = [np.empty((268,)) for _ in range(4)]
AIC_ALL_resp[:], AIC_ALL_resp_traits[:],  AIC_ALL_resp_traits_nosubID[:], AIC_ALL_resp_traits_nosubID_notraits[:] = [np.nan]*4'''

In [ ]:
# RESPTYPE VS RESPTYPE+TRAIT BASED
load_all = 0 # 1: load saved coefs, 0: run the LME
model1_res = []
model2_res = []
model3_res = []
model4_res = []

if load_all == 1:
     df_all_coefs = pd.read_csv(os.path.join(data_file_loc,'coefs_run_norm','slope_reg','LME_S_NS_revision1','lmecoefs_all_S_NS_S_NSwithtraits.csv'))
     #[coef_ALL_stim,pval_ALL_stim] = np.load(os.path.join(data_file_loc,'coefs_run_norm','slope_reg','LME_S_NS_revision1','lmecoefs_all_S_NS_stimType.npy'), allow_pickle=True)
else:
    start_time =  time.time()
    for n in range(268):
        if n%10 == 0:
            print(f'node {n+1},time={(time.time()-start_time)/60:.2f} mins')
        
        df = pd.DataFrame(columns = ['Subject','coefs','subID','movie'])
        for m in range(10):
            df_temp = pd.DataFrame({'Subject':sub_id_all[subs_10resp], 'coefs': all_coefs[subs_10resp,n,m], 'response':responses[subs_10resp,m],\
                'subID':subs_10resp,\
                 'movie':np.repeat(m,len(subs_10resp))})
            df = df.append(df_temp,ignore_index=True)
            df = df.loc[df['response'] != 9] # yes v no
        
        df.set_index("Subject", inplace=True)
        df_both = df.join(res_behav_data.loc[:,['ASR_Intn_T']], how='inner') # join betas and trait info
        inds = ~np.isnan(df_both['coefs']) & ~np.isnan(df_both['ASR_Intn_T']) # find rows where neither x or y is NaN
        df_both = df_both.loc[inds,:]

        nresps_per_sub = df_both.groupby(['subID','response']).count().unstack(level=1).iloc[:,:2]
        nresps_per_sub.columns = nresps_per_sub.columns.droplevel()

        biased_sub_IDs1 = list(nresps_per_sub[nresps_per_sub[0].isna()].index)
        biased_sub_IDs1 # No "Non-social response"

        biased_sub_IDs2 = list(nresps_per_sub[nresps_per_sub[1].isna()].index)
        biased_sub_IDs2 #  No "Social" response

        biased_sub_IDs = biased_sub_IDs1 + biased_sub_IDs2

        subID = df_both['subID'].values
        rows_to_delete = np.zeros((subID.shape[0],))
        for i in biased_sub_IDs:
            rows_to_delete[subID == i] = 1

        df_both['rows_to_delete'] = rows_to_delete
        df_both_nobiasedsubs = df_both.loc[df_both['rows_to_delete']==0,:]
        #print(df_both_nobiasedsubs.shape, df_both.shape)

        #data_both = df.join(res_behav_data.loc[:,['ASR_Intn_T','Age_in_Yrs','Gender']], how='inner') # join betas and trait info

        #df_both['response'] = df_both['response'].map({1.0: "Non-social", 0.0:  "Social"})
        df_both_nobiasedsubs['response'] = df_both_nobiasedsubs['response'].map({1.0: "Non-social", 0.0:  "Social"})
        

        model1 = Lmer('coefs ~  response +(1|subID) + (1|movie)', data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
        model1.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)
        model1_res.append([model1.coefs['Estimate'],model1.coefs['P-val'],model1.AIC])

        model2 = Lmer('coefs ~  response*ASR_Intn_T + (1|movie)', data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
        model2.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)
        model2_res.append([model2.coefs['Estimate'],model2.coefs['P-val'],model2.AIC])

        model3 = Lmer('coefs ~  response + ASR_Intn_T + (1|movie)', data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
        model3.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)
        model3_res.append([model3.coefs['Estimate'],model3.coefs['P-val'],model3.AIC])

        model4 = Lmer('coefs ~  response + (1|movie)', data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
        model4.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)
        model4_res.append([model4.coefs['Estimate'],model4.coefs['P-val'],model4.AIC])

        model5 = Lmer('coefs ~  response + ASR_Intn_T', data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
        model5.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)
        model5_res.append([model4.coefs['Estimate'],model4.coefs['P-val'],model4.AIC])

        model6 = Lmer('coefs ~  response + ASR_Intn_T', data=df_both_nobiasedsubs) # ff: mean response, rf:subjID
        model5.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)
        model5_res.append([model4.coefs['Estimate'],model4.coefs['P-val'],model4.AIC])

        #if n == 0:
        #    print('model:',model1.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False))
        #    print('model:',model2.fit(factors={"response":["Non-social","Social"]}))

        '''coef_ALL_resp[n] = model1.coefs['Estimate'][1]
        pval_ALL_resp[n]= model1.coefs['P-val'][1]
        AIC_ALL_resp[n]= model1.AIC

        coef_ALL_resp_traits[n] = model2.coefs['Estimate'][1]
        pval_ALL_resp_traits[n]= model2.coefs['P-val'][1]
        AIC_ALL_resp_traits[n] = model2.AIC

        coef_ALL_resp_traits_nosubID[n] = model3.coefs['Estimate'][1]
        pval_ALL_resp_traits_nosubID[n]= model3.coefs['P-val'][1]
        AIC_ALL_resp_traits_nosubID[n] = model3.AIC
        
        coef_ALL_resp_traits_nosubID_notraits[n] = model4.coefs['Estimate'][1]
        pval_ALL_resp_traits_nosubID_notraits[n]= model4.coefs['P-val'][1]
        AIC_ALL_resp_traits_nosubID_notraits[n] = model4.AIC'''
        
    '''df_all_coefs = pd.DataFrame({'Node': np.arange(268), 'coef_ALL_resp': coef_ALL_resp, 'pval_ALL_resp': pval_ALL_resp, 'AIC_ALL_resp': AIC_ALL_resp,\
        'coef_ALL_resp_traits': coef_ALL_resp_traits,   'pval_ALL_resp_traits': pval_ALL_resp_traits, 'AIC_ALL_resp_traits': AIC_ALL_resp_traits, \
        'coef_ALL_resp_traits_nosubID': coef_ALL_resp_traits_nosubID, 'pval_ALL_resp_traits_nosubID': pval_ALL_resp_traits_nosubID, 
        'AIC_ALL_resp_traits_nosubID': AIC_ALL_resp_traits_nosubID,
        'coef_ALL_resp_traits_nosubID_notraits': coef_ALL_resp_traits_nosubID_notraits, 'pval_ALL_resp_traits_nosubID_notraits': pval_ALL_resp_traits_nosubID_notraits, 
        'AIC_ALL_resp_traits_nosubID_notraits': AIC_ALL_resp_traits_nosubID_notraits})'''
    #df_all_coefs.to_csv(os.path.join(data_file_loc,'coefs_run_norm','slope_reg','LME_S_NS_revision1','lmecoefs_all_S_NS_S_NSwithtraits.csv'))
    
    print('Done on/at:',datetime.now()) # cell run at)